```
--- Day 12: Hill Climbing Algorithm ---
You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:

v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?

```

In [1]:
import numpy as np
from dataclasses import dataclass
from queue import PriorityQueue
# from tqdm.auto import tqdm

data = np.array([[ord(v) for v in row] for row in open('inputs/12.input').read().splitlines()])
data_max = np.ones(np.array(data.shape)+2)+np.inf
data_max[1:-1,1:-1] = data

init = np.array((data_max == ord('S')).nonzero(),dtype=int).flatten()
data_max[tuple(init)] = ord('a')
target = np.array((data_max == ord('E')).nonzero(),dtype=int).flatten()
data_max[tuple(target)] = ord('z')

def dest_dist(from_pos,to_pos_list):
    return np.abs(to_pos_list-from_pos).sum(1).min()

@dataclass
class Path:
    pos: np.ndarray
    target: np.ndarray
    g: int = 0

    @property
    def h(self):
        return dest_dist(self.pos,self.target)
    
    @property
    def f(self):
        return self.g+self.h


def get_path(from_pos,to_pos_list, data_max):
    visited = []
    q = PriorityQueue()
    counter = 0
    
    def add_q(path):
        nonlocal q, counter
        q.put((path.f,counter,path))
        counter += 1
        
    def get_q():
        return q.get()[2]

    current = Path(from_pos,to_pos_list)
    add_q(current)


    # pb = tqdm(total=data.shape[0]*data.shape[1])
    while not q.empty():
        current = get_q()
        if tuple(current.pos) in visited: continue
        # pb.update()
        
        visited.append(tuple(current.pos))
        
        if current.h == 0:
            break
        
        for step in [[0,1],[0,-1],[1,0],[-1,0]]:
            next_pos = current.pos + step
            if tuple(next_pos) in visited: continue
            
            step_cond = data_max[tuple(next_pos)] - data_max[tuple(current.pos)] < 2
            step_weight = 1 if step_cond else np.inf
            next = Path(next_pos, to_pos_list, g=current.g+step_weight)
            add_q(next)
    
    return current if (current.h == 0).all() else None

In [2]:
get_path(init,target[None,:],data_max).g

383

```
--- Part Two ---
As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:

Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:

...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^
This path reaches the goal in only 29 steps, the fewest possible.

What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?
```

In [3]:
lowest = data_max == ord('a')
candidate_dest = np.array(lowest.nonzero()).T
reversed_map = np.abs(data_max-ord('z'))
get_path(target,candidate_dest,reversed_map).g

377